# TSMixer

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import time
from random import randrange
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import StandardScaler

# Loading Data

In [ ]:
class DataLoader:

  def __init__(self, batch_size, seq_len, pred_len):
    self.batch_size = batch_size
    self.seq_len = seq_len
    self.pred_len = pred_len
    self.target_slice = slice(0, None)

    self._read_data()

  def _read_data(self):

    filepath = ('SP100_vol (original).xlsx')

    df_raw = pd.read_excel(filepath)
    df = df_raw.set_index('Date')

    # split train/valid/test
    n = len(df)
    train_end = int(n * 0.5)
    val_end = int(n * 0.7)
    test_end = n

    train_df = df[:train_end]
    val_df = df[train_end - self.seq_len : val_end]
    test_df = df[val_end - self.seq_len : test_end]

    # standardize by training set
    self.scaler = StandardScaler()
    self.scaler.fit(train_df.values)

    def scale_df(df, scaler):
      data = scaler.transform(df.values)
      return pd.DataFrame(data, index=df.index, columns=df.columns)

    self.train_df = scale_df(train_df, self.scaler)
    self.val_df = scale_df(val_df, self.scaler)
    self.test_df = scale_df(test_df, self.scaler)
    self.n_feature = self.train_df.shape[-1]

  def _split_window(self, data):
    inputs = data[:, : self.seq_len, :]
    labels = data[:, self.seq_len :, self.target_slice]

    inputs.set_shape([None, self.seq_len, None])
    labels.set_shape([None, self.pred_len, None])
    return inputs, labels

  def _make_dataset(self, data, shuffle=False):
    data = np.array(data, dtype=np.float32)
    ds = tf.keras.utils.timeseries_dataset_from_array(
        data=data,
        targets=None,
        sequence_length=(self.seq_len + self.pred_len),
        sequence_stride=1,
        shuffle=shuffle,
        batch_size=self.batch_size,
    )
    ds = ds.map(self._split_window)
    return ds

  def inverse_transform(self, data):
    return self.scaler.inverse_transform(data)

  def get_train(self, shuffle=True):
    return self._make_dataset(self.train_df, shuffle=shuffle)

  def get_val(self):
    return self._make_dataset(self.val_df, shuffle=False)

  def get_test(self):
    return self._make_dataset(self.test_df, shuffle=False)

  def make_forecast(self, model):
    test_data = self.test_df.values
    predictions = []

    for i in range(0, len(test_data) - self.seq_len, self.pred_len):
        input_seq = test_data[i:i + self.seq_len]
        input_seq = input_seq.reshape((1, self.seq_len, -1))

            # Make prediction and reshape output
        pred = model.predict(input_seq)
        pred = pred.reshape(self.pred_len, -1)

        predictions.append(pred)

        # Concatenate all predictions
    predictions = np.concatenate(predictions, axis=0)
    return predictions

In [ ]:
    filepath = ('SP100_vol (original).xlsx')

    df_raw = pd.read_excel(filepath)
    df = df_raw.set_index('Date')

    # split train/valid/test
    n = len(df)
    train_end = int(n * 0.5)
    val_end = int(n * 0.7)
    test_end = n

test_true=df[val_end - n_inputs[a2[i]] : test_end]
test_true=test_true.iloc[n_inputs[a2[i]]:]

# Hyperparameter Search

In [ ]:
dropout=[0.7,0.75,0.8,0.85,0.9]
n_inputs = [3,5,7,10,15,21]
n_blocks=[2,3,4,5,6]
ff_dim=[160,180,220,260,300,320,400]
learning_rate=[0.0005,0.0001,0.00005,0.00001]
epochs=[150,250,350,450,550]

random_seeds=[]
for i in range(60):
  random_seeds.append(randrange(129228148))

a1=[]
a2=[]
a3=[]
a4=[]
a5=[]
a6=[]
for i in range(40):
  a1.append(randrange(5))
  a2.append(randrange(6))
  a3.append(randrange(5))
  a4.append(randrange(7))
  a5.append(randrange(4))
  a6.append(randrange(5))

In [ ]:
RMSE =[]
MAE =[]
MAPE = []
QLIKE = []

for i in range(5):
  data_loader = DataLoader(batch_size=64, seq_len=n_inputs[a2[i]], pred_len=1)

  train_data = data_loader.get_train()
  val_data = data_loader.get_val()
  test_data = data_loader.get_test()

  def res_block(inputs, ff_dim):

    norm = layers.BatchNormalization

    # Time mixing
    x = norm(axis=[-2, -1])(inputs)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(x.shape[-1], activation='relu')(x)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Input Length, Channel]
    x = layers.Dropout(dropout[a1[i]])(x)
    res = x + inputs

    # Feature mixing
    x = norm(axis=[-2, -1])(res)
    x = layers.Dense(ff_dim, activation='relu')(x)  # [Batch, Input Length, FF_Dim]
    x = layers.Dropout(dropout[a1[i]])(x)
    x = layers.Dense(inputs.shape[-1])(x)  # [Batch, Input Length, Channel]
    x = layers.Dropout(dropout[a1[i]])(x)
    return x + res

  def build_model(
      input_shape,
      pred_len,
      n_block,
      ff_dim,
      target_slice,
  ):

    inputs = tf.keras.Input(shape=input_shape)
    x = inputs  # [Batch, Input Length, Channel]
    for _ in range(n_block):
      x = res_block(x, ff_dim)

    if target_slice:
      x = x[:, :, target_slice]

    # Temporal projection
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(pred_len)(x)  # [Batch, Channel, Output Length]
    outputs = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Output Length, Channel])

    return tf.keras.Model(inputs, outputs)

  model = build_model(
      input_shape=(n_inputs[a2[i]], data_loader.n_feature),
      pred_len=1,
      n_block=n_blocks[a3[i]],
      ff_dim=ff_dim[a4[i]],
      target_slice=data_loader.target_slice
  )

  tf.keras.utils.set_random_seed(random_seeds[i])

  optimizer = tf.keras.optimizers.Adam(learning_rate[a5[i]])

  model.compile(optimizer, loss='mse', metrics=['mae', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error'])

  early_stop_callback = tf.keras.callbacks.EarlyStopping(
      monitor='val_loss',
      patience=5
  )

  start_training_time = time.time()

  history = model.fit(
      train_data,
      epochs= epochs[a6[i]],
      validation_data=val_data,
      callbacks=[early_stop_callback]
  )

  end_training_time = time.time()
  elasped_training_time = end_training_time - start_training_time
  print(f'Training finished in {elasped_training_time} seconds')
  RMSE.append(np.sqrt(history.history['val_loss'][np.argmin(history.history['val_loss'])]))
  MAE.append(history.history['val_mae'][np.argmin(history.history['val_mae'])])
  MAPE.append(history.history['val_mean_absolute_percentage_error'][np.argmin(history.history['val_mean_absolute_percentage_error'])])
  QLIKE.append(history.history['val_mean_squared_logarithmic_error'][np.argmin(history.history['val_mean_squared_logarithmic_error'])])

Epoch 1/250
27/27 [==============================] - 12s 56ms/step - loss: 191.2941 - mae: 7.1503 - mean_absolute_percentage_error: 7425.2593 - mean_squared_logarithmic_error: 1.2799 - val_loss: 1.2017 - val_mae: 0.9277 - val_mean_absolute_percentage_error: 279.7266 - val_mean_squared_logarithmic_error: 0.0856
Epoch 2/250
27/27 [==============================] - 1s 26ms/step - loss: 166.6711 - mae: 6.6987 - mean_absolute_percentage_error: 5977.5684 - mean_squared_logarithmic_error: 1.2342 - val_loss: 1.4749 - val_mae: 1.0461 - val_mean_absolute_percentage_error: 319.0156 - val_mean_squared_logarithmic_error: 0.1371
Epoch 3/250
27/27 [==============================] - 1s 26ms/step - loss: 143.4553 - mae: 6.2651 - mean_absolute_percentage_error: 4933.6533 - mean_squared_logarithmic_error: 1.1993 - val_loss: 1.6155 - val_mae: 1.1040 - val_mean_absolute_percentage_error: 336.6630 - val_mean_squared_logarithmic_error: 0.1651
Epoch 4/250
27/27 [==============================] - 1s 27ms/step 

In [ ]:
for i in range(5):
  i=i+5*1
  data_loader = DataLoader(batch_size=32, seq_len=n_inputs[a2[i]], pred_len=1)

  train_data = data_loader.get_train()
  val_data = data_loader.get_val()
  test_data = data_loader.get_test()

  def res_block(inputs, ff_dim):

    norm = layers.BatchNormalization

    # Time mixing
    x = norm(axis=[-2, -1])(inputs)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(x.shape[-1], activation='relu')(x)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Input Length, Channel]
    x = layers.Dropout(dropout[a1[i]])(x)
    res = x + inputs

    # Feature mixing
    x = norm(axis=[-2, -1])(res)
    x = layers.Dense(ff_dim, activation='relu')(x)  # [Batch, Input Length, FF_Dim]
    x = layers.Dropout(dropout[a1[i]])(x)
    x = layers.Dense(inputs.shape[-1])(x)  # [Batch, Input Length, Channel]
    x = layers.Dropout(dropout[a1[i]])(x)
    return x + res

  def build_model(
      input_shape,
      pred_len,
      n_block,
      ff_dim,
      target_slice,
  ):

    inputs = tf.keras.Input(shape=input_shape)
    x = inputs  # [Batch, Input Length, Channel]
    for _ in range(n_block):
      x = res_block(x, ff_dim)

    if target_slice:
      x = x[:, :, target_slice]

    # Temporal projection
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(pred_len)(x)  # [Batch, Channel, Output Length]
    outputs = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Output Length, Channel])

    return tf.keras.Model(inputs, outputs)

  model = build_model(
      input_shape=(n_inputs[a2[i]], data_loader.n_feature),
      pred_len=1,
      n_block=n_blocks[a3[i]],
      ff_dim=ff_dim[a4[i]],
      target_slice=data_loader.target_slice
  )

  tf.keras.utils.set_random_seed(random_seeds[i])

  optimizer = tf.keras.optimizers.Adam(learning_rate[a5[i]])

  model.compile(optimizer, loss='mse', metrics=['mae', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error'])

  early_stop_callback = tf.keras.callbacks.EarlyStopping(
      monitor='val_loss',
      patience=5
  )

  start_training_time = time.time()

  history = model.fit(
      train_data,
      epochs= epochs[a6[i]],
      validation_data=val_data,
      callbacks=[early_stop_callback]
  )

  end_training_time = time.time()
  elasped_training_time = end_training_time - start_training_time
  print(f'Training finished in {elasped_training_time} seconds')
  RMSE.append(np.sqrt(history.history['val_loss'][np.argmin(history.history['val_loss'])]))
  MAE.append(history.history['val_mae'][np.argmin(history.history['val_mae'])])
  MAPE.append(history.history['val_mean_absolute_percentage_error'][np.argmin(history.history['val_mean_absolute_percentage_error'])])
  QLIKE.append(history.history['val_mean_squared_logarithmic_error'][np.argmin(history.history['val_mean_squared_logarithmic_error'])])

Epoch 1/350
53/53 [==============================] - 7s 29ms/step - loss: 251.7316 - mae: 11.6693 - mean_absolute_percentage_error: 10438.7207 - mean_squared_logarithmic_error: 2.3269 - val_loss: 1.8874 - val_mae: 1.0819 - val_mean_absolute_percentage_error: 489.1088 - val_mean_squared_logarithmic_error: 0.1819
Epoch 2/350
53/53 [==============================] - 1s 22ms/step - loss: 248.6531 - mae: 11.6182 - mean_absolute_percentage_error: 13691.6250 - mean_squared_logarithmic_error: 2.3162 - val_loss: 1.9358 - val_mae: 1.1159 - val_mean_absolute_percentage_error: 490.6023 - val_mean_squared_logarithmic_error: 0.1992
Epoch 3/350
53/53 [==============================] - 1s 25ms/step - loss: 247.4544 - mae: 11.5898 - mean_absolute_percentage_error: 9575.7930 - mean_squared_logarithmic_error: 2.3162 - val_loss: 1.9467 - val_mae: 1.1268 - val_mean_absolute_percentage_error: 492.6684 - val_mean_squared_logarithmic_error: 0.2043
Epoch 4/350
53/53 [==============================] - 2s 29ms/s

In [ ]:
for i in range(5):
  i=i+5*2
  data_loader = DataLoader(batch_size=32, seq_len=n_inputs[a2[i]], pred_len=1)

  train_data = data_loader.get_train()
  val_data = data_loader.get_val()
  test_data = data_loader.get_test()

  def res_block(inputs, ff_dim):

    norm = layers.BatchNormalization

    # Time mixing
    x = norm(axis=[-2, -1])(inputs)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(x.shape[-1], activation='relu')(x)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Input Length, Channel]
    x = layers.Dropout(dropout[a1[i]])(x)
    res = x + inputs

    # Feature mixing
    x = norm(axis=[-2, -1])(res)
    x = layers.Dense(ff_dim, activation='relu')(x)  # [Batch, Input Length, FF_Dim]
    x = layers.Dropout(dropout[a1[i]])(x)
    x = layers.Dense(inputs.shape[-1])(x)  # [Batch, Input Length, Channel]
    x = layers.Dropout(dropout[a1[i]])(x)
    return x + res

  def build_model(
      input_shape,
      pred_len,
      n_block,
      ff_dim,
      target_slice,
  ):

    inputs = tf.keras.Input(shape=input_shape)
    x = inputs  # [Batch, Input Length, Channel]
    for _ in range(n_block):
      x = res_block(x, ff_dim)

    if target_slice:
      x = x[:, :, target_slice]

    # Temporal projection
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(pred_len)(x)  # [Batch, Channel, Output Length]
    outputs = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Output Length, Channel])

    return tf.keras.Model(inputs, outputs)

  model = build_model(
      input_shape=(n_inputs[a2[i]], data_loader.n_feature),
      pred_len=1,
      n_block=n_blocks[a3[i]],
      ff_dim=ff_dim[a4[i]],
      target_slice=data_loader.target_slice
  )

  tf.keras.utils.set_random_seed(random_seeds[i])

  optimizer = tf.keras.optimizers.Adam(learning_rate[a5[i]])

  model.compile(optimizer, loss='mse', metrics=['mae', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error'])

  early_stop_callback = tf.keras.callbacks.EarlyStopping(
      monitor='val_loss',
      patience=5
  )

  start_training_time = time.time()

  history = model.fit(
      train_data,
      epochs= epochs[a6[i]],
      validation_data=val_data,
      callbacks=[early_stop_callback]
  )

  end_training_time = time.time()
  elasped_training_time = end_training_time - start_training_time
  print(f'Training finished in {elasped_training_time} seconds')
  RMSE.append(np.sqrt(history.history['val_loss'][np.argmin(history.history['val_loss'])]))
  MAE.append(history.history['val_mae'][np.argmin(history.history['val_mae'])])
  MAPE.append(history.history['val_mean_absolute_percentage_error'][np.argmin(history.history['val_mean_absolute_percentage_error'])])
  QLIKE.append(history.history['val_mean_squared_logarithmic_error'][np.argmin(history.history['val_mean_squared_logarithmic_error'])])

Epoch 1/150
54/54 [==============================] - 7s 30ms/step - loss: 45.8816 - mae: 4.5753 - mean_absolute_percentage_error: 3776.7490 - mean_squared_logarithmic_error: 1.2351 - val_loss: 1.0799 - val_mae: 0.7181 - val_mean_absolute_percentage_error: 439.1298 - val_mean_squared_logarithmic_error: 0.1104
Epoch 2/150
54/54 [==============================] - 1s 21ms/step - loss: 45.8515 - mae: 4.5374 - mean_absolute_percentage_error: 4239.8228 - mean_squared_logarithmic_error: 1.2206 - val_loss: 0.9755 - val_mae: 0.6822 - val_mean_absolute_percentage_error: 428.1679 - val_mean_squared_logarithmic_error: 0.0990
Epoch 3/150
54/54 [==============================] - 1s 18ms/step - loss: 45.4100 - mae: 4.5391 - mean_absolute_percentage_error: 4985.0298 - mean_squared_logarithmic_error: 1.2198 - val_loss: 0.9378 - val_mae: 0.6702 - val_mean_absolute_percentage_error: 424.6649 - val_mean_squared_logarithmic_error: 0.0949
Epoch 4/150
54/54 [==============================] - 1s 19ms/step - lo

In [ ]:
for i in range(5):
  i=i+5*3
  data_loader = DataLoader(batch_size=32, seq_len=n_inputs[a2[i]], pred_len=1)

  train_data = data_loader.get_train()
  val_data = data_loader.get_val()
  test_data = data_loader.get_test()

  def res_block(inputs, ff_dim):

    norm = layers.BatchNormalization

    # Time mixing
    x = norm(axis=[-2, -1])(inputs)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(x.shape[-1], activation='relu')(x)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Input Length, Channel]
    x = layers.Dropout(dropout[a1[i]])(x)
    res = x + inputs

    # Feature mixing
    x = norm(axis=[-2, -1])(res)
    x = layers.Dense(ff_dim, activation='relu')(x)  # [Batch, Input Length, FF_Dim]
    x = layers.Dropout(dropout[a1[i]])(x)
    x = layers.Dense(inputs.shape[-1])(x)  # [Batch, Input Length, Channel]
    x = layers.Dropout(dropout[a1[i]])(x)
    return x + res

  def build_model(
      input_shape,
      pred_len,
      n_block,
      ff_dim,
      target_slice,
  ):

    inputs = tf.keras.Input(shape=input_shape)
    x = inputs  # [Batch, Input Length, Channel]
    for _ in range(n_block):
      x = res_block(x, ff_dim)

    if target_slice:
      x = x[:, :, target_slice]

    # Temporal projection
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(pred_len)(x)  # [Batch, Channel, Output Length]
    outputs = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Output Length, Channel])

    return tf.keras.Model(inputs, outputs)

  model = build_model(
      input_shape=(n_inputs[a2[i]], data_loader.n_feature),
      pred_len=1,
      n_block=n_blocks[a3[i]],
      ff_dim=ff_dim[a4[i]],
      target_slice=data_loader.target_slice
  )

  tf.keras.utils.set_random_seed(random_seeds[i])

  optimizer = tf.keras.optimizers.Adam(learning_rate[a5[i]])

  model.compile(optimizer, loss='mse', metrics=['mae', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error'])

  early_stop_callback = tf.keras.callbacks.EarlyStopping(
      monitor='val_loss',
      patience=5
  )

  start_training_time = time.time()

  history = model.fit(
      train_data,
      epochs= epochs[a6[i]],
      validation_data=val_data,
      callbacks=[early_stop_callback]
  )

  end_training_time = time.time()
  elasped_training_time = end_training_time - start_training_time
  print(f'Training finished in {elasped_training_time} seconds')
  RMSE.append(np.sqrt(history.history['val_loss'][np.argmin(history.history['val_loss'])]))
  MAE.append(history.history['val_mae'][np.argmin(history.history['val_mae'])])
  MAPE.append(history.history['val_mean_absolute_percentage_error'][np.argmin(history.history['val_mean_absolute_percentage_error'])])
  QLIKE.append(history.history['val_mean_squared_logarithmic_error'][np.argmin(history.history['val_mean_squared_logarithmic_error'])])

Epoch 1/450
54/54 [==============================] - 7s 30ms/step - loss: 43.4914 - mae: 4.6969 - mean_absolute_percentage_error: 4943.4365 - mean_squared_logarithmic_error: 1.3168 - val_loss: 0.7545 - val_mae: 0.6340 - val_mean_absolute_percentage_error: 360.0196 - val_mean_squared_logarithmic_error: 0.0848
Epoch 2/450
54/54 [==============================] - 1s 27ms/step - loss: 28.8630 - mae: 3.8788 - mean_absolute_percentage_error: 4172.3774 - mean_squared_logarithmic_error: 1.0888 - val_loss: 0.6518 - val_mae: 0.6134 - val_mean_absolute_percentage_error: 317.5988 - val_mean_squared_logarithmic_error: 0.0708
Epoch 3/450
54/54 [==============================] - 1s 24ms/step - loss: 21.1369 - mae: 3.3204 - mean_absolute_percentage_error: 4486.9990 - mean_squared_logarithmic_error: 0.9330 - val_loss: 0.6125 - val_mae: 0.6133 - val_mean_absolute_percentage_error: 296.9197 - val_mean_squared_logarithmic_error: 0.0619
Epoch 4/450
54/54 [==============================] - 1s 23ms/step - lo

In [ ]:
for i in range(5):
  i=i+5*4
  data_loader = DataLoader(batch_size=32, seq_len=n_inputs[a2[i]], pred_len=1)

  train_data = data_loader.get_train()
  val_data = data_loader.get_val()
  test_data = data_loader.get_test()

  def res_block(inputs, ff_dim):

    norm = layers.BatchNormalization

    # Time mixing
    x = norm(axis=[-2, -1])(inputs)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(x.shape[-1], activation='relu')(x)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Input Length, Channel]
    x = layers.Dropout(dropout[a1[i]])(x)
    res = x + inputs

    # Feature mixing
    x = norm(axis=[-2, -1])(res)
    x = layers.Dense(ff_dim, activation='relu')(x)  # [Batch, Input Length, FF_Dim]
    x = layers.Dropout(dropout[a1[i]])(x)
    x = layers.Dense(inputs.shape[-1])(x)  # [Batch, Input Length, Channel]
    x = layers.Dropout(dropout[a1[i]])(x)
    return x + res

  def build_model(
      input_shape,
      pred_len,
      n_block,
      ff_dim,
      target_slice,
  ):

    inputs = tf.keras.Input(shape=input_shape)
    x = inputs  # [Batch, Input Length, Channel]
    for _ in range(n_block):
      x = res_block(x, ff_dim)

    if target_slice:
      x = x[:, :, target_slice]

    # Temporal projection
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(pred_len)(x)  # [Batch, Channel, Output Length]
    outputs = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Output Length, Channel])

    return tf.keras.Model(inputs, outputs)

  model = build_model(
      input_shape=(n_inputs[a2[i]], data_loader.n_feature),
      pred_len=1,
      n_block=n_blocks[a3[i]],
      ff_dim=ff_dim[a4[i]],
      target_slice=data_loader.target_slice
  )

  tf.keras.utils.set_random_seed(random_seeds[i])

  optimizer = tf.keras.optimizers.Adam(learning_rate[a5[i]])

  model.compile(optimizer, loss='mse', metrics=['mae', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error'])

  early_stop_callback = tf.keras.callbacks.EarlyStopping(
      monitor='val_loss',
      patience=5
  )

  start_training_time = time.time()

  history = model.fit(
      train_data,
      epochs= epochs[a6[i]],
      validation_data=val_data,
      callbacks=[early_stop_callback]
  )

  end_training_time = time.time()
  elasped_training_time = end_training_time - start_training_time
  print(f'Training finished in {elasped_training_time} seconds')
  RMSE.append(np.sqrt(history.history['val_loss'][np.argmin(history.history['val_loss'])]))
  MAE.append(history.history['val_mae'][np.argmin(history.history['val_mae'])])
  MAPE.append(history.history['val_mean_absolute_percentage_error'][np.argmin(history.history['val_mean_absolute_percentage_error'])])
  QLIKE.append(history.history['val_mean_squared_logarithmic_error'][np.argmin(history.history['val_mean_squared_logarithmic_error'])])

Epoch 1/250
53/53 [==============================] - 10s 32ms/step - loss: 52.1927 - mae: 5.1417 - mean_absolute_percentage_error: 4307.1680 - mean_squared_logarithmic_error: 0.9259 - val_loss: 1.2187 - val_mae: 0.7610 - val_mean_absolute_percentage_error: 621.0471 - val_mean_squared_logarithmic_error: 0.0499
Epoch 2/250
53/53 [==============================] - 1s 23ms/step - loss: 51.0890 - mae: 5.0701 - mean_absolute_percentage_error: 4865.9443 - mean_squared_logarithmic_error: 0.9224 - val_loss: 0.9955 - val_mae: 0.6898 - val_mean_absolute_percentage_error: 568.4821 - val_mean_squared_logarithmic_error: 0.0508
Epoch 3/250
53/53 [==============================] - 1s 23ms/step - loss: 50.3998 - mae: 5.0553 - mean_absolute_percentage_error: 6155.2153 - mean_squared_logarithmic_error: 0.9235 - val_loss: 0.9289 - val_mae: 0.6699 - val_mean_absolute_percentage_error: 550.5568 - val_mean_squared_logarithmic_error: 0.0518
Epoch 4/250
53/53 [==============================] - 2s 28ms/step - l

In [ ]:
for i in range(5):
  i=i+5*5
  data_loader = DataLoader(batch_size=32, seq_len=n_inputs[a2[i]], pred_len=1)

  train_data = data_loader.get_train()
  val_data = data_loader.get_val()
  test_data = data_loader.get_test()

  def res_block(inputs, ff_dim):

    norm = layers.BatchNormalization

    # Time mixing
    x = norm(axis=[-2, -1])(inputs)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(x.shape[-1], activation='relu')(x)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Input Length, Channel]
    x = layers.Dropout(dropout[a1[i]])(x)
    res = x + inputs

    # Feature mixing
    x = norm(axis=[-2, -1])(res)
    x = layers.Dense(ff_dim, activation='relu')(x)  # [Batch, Input Length, FF_Dim]
    x = layers.Dropout(dropout[a1[i]])(x)
    x = layers.Dense(inputs.shape[-1])(x)  # [Batch, Input Length, Channel]
    x = layers.Dropout(dropout[a1[i]])(x)
    return x + res

  def build_model(
      input_shape,
      pred_len,
      n_block,
      ff_dim,
      target_slice,
  ):

    inputs = tf.keras.Input(shape=input_shape)
    x = inputs  # [Batch, Input Length, Channel]
    for _ in range(n_block):
      x = res_block(x, ff_dim)

    if target_slice:
      x = x[:, :, target_slice]

    # Temporal projection
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(pred_len)(x)  # [Batch, Channel, Output Length]
    outputs = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Output Length, Channel])

    return tf.keras.Model(inputs, outputs)

  model = build_model(
      input_shape=(n_inputs[a2[i]], data_loader.n_feature),
      pred_len=1,
      n_block=n_blocks[a3[i]],
      ff_dim=ff_dim[a4[i]],
      target_slice=data_loader.target_slice
  )

  tf.keras.utils.set_random_seed(random_seeds[i])

  optimizer = tf.keras.optimizers.Adam(learning_rate[a5[i]])

  model.compile(optimizer, loss='mse', metrics=['mae', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error'])

  early_stop_callback = tf.keras.callbacks.EarlyStopping(
      monitor='val_loss',
      patience=5
  )

  start_training_time = time.time()

  history = model.fit(
      train_data,
      epochs= epochs[a6[i]],
      validation_data=val_data,
      callbacks=[early_stop_callback]
  )

  end_training_time = time.time()
  elasped_training_time = end_training_time - start_training_time
  print(f'Training finished in {elasped_training_time} seconds')
  RMSE.append(np.sqrt(history.history['val_loss'][np.argmin(history.history['val_loss'])]))
  MAE.append(history.history['val_mae'][np.argmin(history.history['val_mae'])])
  MAPE.append(history.history['val_mean_absolute_percentage_error'][np.argmin(history.history['val_mean_absolute_percentage_error'])])
  QLIKE.append(history.history['val_mean_squared_logarithmic_error'][np.argmin(history.history['val_mean_squared_logarithmic_error'])])

Epoch 1/350
54/54 [==============================] - 5s 21ms/step - loss: 60.0283 - mae: 5.4535 - mean_absolute_percentage_error: 6699.0864 - mean_squared_logarithmic_error: 1.3046 - val_loss: 0.8003 - val_mae: 0.6286 - val_mean_absolute_percentage_error: 471.2773 - val_mean_squared_logarithmic_error: 0.0838
Epoch 2/350
54/54 [==============================] - 1s 15ms/step - loss: 34.3355 - mae: 4.2087 - mean_absolute_percentage_error: 5158.0747 - mean_squared_logarithmic_error: 1.0495 - val_loss: 0.7111 - val_mae: 0.6008 - val_mean_absolute_percentage_error: 411.1584 - val_mean_squared_logarithmic_error: 0.0778
Epoch 3/350
54/54 [==============================] - 1s 14ms/step - loss: 24.7094 - mae: 3.5719 - mean_absolute_percentage_error: 4012.7498 - mean_squared_logarithmic_error: 0.9143 - val_loss: 0.6946 - val_mae: 0.6143 - val_mean_absolute_percentage_error: 380.4990 - val_mean_squared_logarithmic_error: 0.0740
Epoch 4/350
54/54 [==============================] - 1s 15ms/step - lo

In [ ]:
for i in range(5):
  i=i+5*6
  data_loader = DataLoader(batch_size=32, seq_len=n_inputs[a2[i]], pred_len=1)

  train_data = data_loader.get_train()
  val_data = data_loader.get_val()
  test_data = data_loader.get_test()

  def res_block(inputs, ff_dim):

    norm = layers.BatchNormalization

    # Time mixing
    x = norm(axis=[-2, -1])(inputs)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(x.shape[-1], activation='relu')(x)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Input Length, Channel]
    x = layers.Dropout(dropout[a1[i]])(x)
    res = x + inputs

    # Feature mixing
    x = norm(axis=[-2, -1])(res)
    x = layers.Dense(ff_dim, activation='relu')(x)  # [Batch, Input Length, FF_Dim]
    x = layers.Dropout(dropout[a1[i]])(x)
    x = layers.Dense(inputs.shape[-1])(x)  # [Batch, Input Length, Channel]
    x = layers.Dropout(dropout[a1[i]])(x)
    return x + res

  def build_model(
      input_shape,
      pred_len,
      n_block,
      ff_dim,
      target_slice,
  ):

    inputs = tf.keras.Input(shape=input_shape)
    x = inputs  # [Batch, Input Length, Channel]
    for _ in range(n_block):
      x = res_block(x, ff_dim)

    if target_slice:
      x = x[:, :, target_slice]

    # Temporal projection
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(pred_len)(x)  # [Batch, Channel, Output Length]
    outputs = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Output Length, Channel])

    return tf.keras.Model(inputs, outputs)

  model = build_model(
      input_shape=(n_inputs[a2[i]], data_loader.n_feature),
      pred_len=1,
      n_block=n_blocks[a3[i]],
      ff_dim=ff_dim[a4[i]],
      target_slice=data_loader.target_slice
  )

  tf.keras.utils.set_random_seed(random_seeds[i])

  optimizer = tf.keras.optimizers.Adam(learning_rate[a5[i]])

  model.compile(optimizer, loss='mse', metrics=['mae', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error'])

  early_stop_callback = tf.keras.callbacks.EarlyStopping(
      monitor='val_loss',
      patience=5
  )

  start_training_time = time.time()

  history = model.fit(
      train_data,
      epochs= epochs[a6[i]],
      validation_data=val_data,
      callbacks=[early_stop_callback]
  )

  end_training_time = time.time()
  elasped_training_time = end_training_time - start_training_time
  print(f'Training finished in {elasped_training_time} seconds')
  RMSE.append(np.sqrt(history.history['val_loss'][np.argmin(history.history['val_loss'])]))
  MAE.append(history.history['val_mae'][np.argmin(history.history['val_mae'])])
  MAPE.append(history.history['val_mean_absolute_percentage_error'][np.argmin(history.history['val_mean_absolute_percentage_error'])])
  QLIKE.append(history.history['val_mean_squared_logarithmic_error'][np.argmin(history.history['val_mean_squared_logarithmic_error'])])

Epoch 1/550
53/53 [==============================] - 12s 43ms/step - loss: 48.7626 - mae: 5.2336 - mean_absolute_percentage_error: 5079.4678 - mean_squared_logarithmic_error: 1.5752 - val_loss: 1.4101 - val_mae: 1.0099 - val_mean_absolute_percentage_error: 365.0668 - val_mean_squared_logarithmic_error: 0.1682
Epoch 2/550
53/53 [==============================] - 1s 27ms/step - loss: 31.4903 - mae: 4.2394 - mean_absolute_percentage_error: 5168.1499 - mean_squared_logarithmic_error: 1.2218 - val_loss: 0.8352 - val_mae: 0.7527 - val_mean_absolute_percentage_error: 294.0747 - val_mean_squared_logarithmic_error: 0.0781
Epoch 3/550
53/53 [==============================] - 1s 27ms/step - loss: 22.2693 - mae: 3.5763 - mean_absolute_percentage_error: 4318.0161 - mean_squared_logarithmic_error: 1.0021 - val_loss: 0.6177 - val_mae: 0.6278 - val_mean_absolute_percentage_error: 269.6046 - val_mean_squared_logarithmic_error: 0.0547
Epoch 4/550
53/53 [==============================] - 2s 29ms/step - l

In [ ]:
for i in range(5):
  i=i+5*7
  data_loader = DataLoader(batch_size=32, seq_len=n_inputs[a2[i]], pred_len=1)

  train_data = data_loader.get_train()
  val_data = data_loader.get_val()
  test_data = data_loader.get_test()

  def res_block(inputs, ff_dim):

    norm = layers.BatchNormalization

    # Time mixing
    x = norm(axis=[-2, -1])(inputs)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(x.shape[-1], activation='relu')(x)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Input Length, Channel]
    x = layers.Dropout(dropout[a1[i]])(x)
    res = x + inputs

    # Feature mixing
    x = norm(axis=[-2, -1])(res)
    x = layers.Dense(ff_dim, activation='relu')(x)  # [Batch, Input Length, FF_Dim]
    x = layers.Dropout(dropout[a1[i]])(x)
    x = layers.Dense(inputs.shape[-1])(x)  # [Batch, Input Length, Channel]
    x = layers.Dropout(dropout[a1[i]])(x)
    return x + res

  def build_model(
      input_shape,
      pred_len,
      n_block,
      ff_dim,
      target_slice,
  ):

    inputs = tf.keras.Input(shape=input_shape)
    x = inputs  # [Batch, Input Length, Channel]
    for _ in range(n_block):
      x = res_block(x, ff_dim)

    if target_slice:
      x = x[:, :, target_slice]

    # Temporal projection
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(pred_len)(x)  # [Batch, Channel, Output Length]
    outputs = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Output Length, Channel])

    return tf.keras.Model(inputs, outputs)

  model = build_model(
      input_shape=(n_inputs[a2[i]], data_loader.n_feature),
      pred_len=1,
      n_block=n_blocks[a3[i]],
      ff_dim=ff_dim[a4[i]],
      target_slice=data_loader.target_slice
  )

  tf.keras.utils.set_random_seed(random_seeds[i])

  optimizer = tf.keras.optimizers.Adam(learning_rate[a5[i]])

  model.compile(optimizer, loss='mse', metrics=['mae', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error'])

  early_stop_callback = tf.keras.callbacks.EarlyStopping(
      monitor='val_loss',
      patience=5
  )

  start_training_time = time.time()

  history = model.fit(
      train_data,
      epochs= epochs[a6[i]],
      validation_data=val_data,
      callbacks=[early_stop_callback]
  )

  end_training_time = time.time()
  elasped_training_time = end_training_time - start_training_time
  print(f'Training finished in {elasped_training_time} seconds')
  RMSE.append(np.sqrt(history.history['val_loss'][np.argmin(history.history['val_loss'])]))
  MAE.append(history.history['val_mae'][np.argmin(history.history['val_mae'])])
  MAPE.append(history.history['val_mean_absolute_percentage_error'][np.argmin(history.history['val_mean_absolute_percentage_error'])])
  QLIKE.append(history.history['val_mean_squared_logarithmic_error'][np.argmin(history.history['val_mean_squared_logarithmic_error'])])

Epoch 1/150
53/53 [==============================] - 12s 41ms/step - loss: 523.7309 - mae: 16.5618 - mean_absolute_percentage_error: 27320.6133 - mean_squared_logarithmic_error: 3.6414 - val_loss: 1.8893 - val_mae: 0.8868 - val_mean_absolute_percentage_error: 566.0685 - val_mean_squared_logarithmic_error: 0.1847
Epoch 2/150
53/53 [==============================] - 1s 27ms/step - loss: 508.3055 - mae: 16.3336 - mean_absolute_percentage_error: 15243.9570 - mean_squared_logarithmic_error: 3.6055 - val_loss: 1.5542 - val_mae: 0.8219 - val_mean_absolute_percentage_error: 526.2531 - val_mean_squared_logarithmic_error: 0.1588
Epoch 3/150
53/53 [==============================] - 2s 41ms/step - loss: 489.1715 - mae: 16.0070 - mean_absolute_percentage_error: 22753.9062 - mean_squared_logarithmic_error: 3.5300 - val_loss: 1.4262 - val_mae: 0.7954 - val_mean_absolute_percentage_error: 511.4273 - val_mean_squared_logarithmic_error: 0.1481
Epoch 4/150
53/53 [==============================] - 1s 28ms

In [ ]:
np.argmin(RMSE)

14

In [ ]:
np.argmin(MAE)

14

In [ ]:
np.argmin(MAPE)

18

In [ ]:
np.argmin(QLIKE)

14

In [ ]:
print(dropout[a1[14]])
print(n_inputs[a2[14]])
print(n_blocks[a3[14]])
print(ff_dim[a4[14]])
print(learning_rate[a5[14]])
print(epochs[a6[14]])

print(random_seeds[14])

0.7
21
2
220
0.0005
250
23011641


# Hyperparameter Search (focused on random seed)

In [ ]:
RMSE =[]
MAE =[]
MAPE = []
QLIKE = []

for i in range(5):
  i=i+5*8
  data_loader = DataLoader(batch_size=32, seq_len=21, pred_len=1)

  train_data = data_loader.get_train()
  val_data = data_loader.get_val()
  test_data = data_loader.get_test()

  def res_block(inputs, ff_dim):

    norm = layers.BatchNormalization

    # Time mixing
    x = norm(axis=[-2, -1])(inputs)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(x.shape[-1], activation='relu')(x)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Input Length, Channel]
    x = layers.Dropout(0.7)(x)
    res = x + inputs

    # Feature mixing
    x = norm(axis=[-2, -1])(res)
    x = layers.Dense(ff_dim, activation='relu')(x)  # [Batch, Input Length, FF_Dim]
    x = layers.Dropout(0.7)(x)
    x = layers.Dense(inputs.shape[-1])(x)  # [Batch, Input Length, Channel]
    x = layers.Dropout(0.7)(x)
    return x + res

  def build_model(
      input_shape,
      pred_len,
      n_block,
      ff_dim,
      target_slice,
  ):

    inputs = tf.keras.Input(shape=input_shape)
    x = inputs  # [Batch, Input Length, Channel]
    for _ in range(n_block):
      x = res_block(x, ff_dim)

    if target_slice:
      x = x[:, :, target_slice]

    # Temporal projection
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(pred_len)(x)  # [Batch, Channel, Output Length]
    outputs = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Output Length, Channel])

    return tf.keras.Model(inputs, outputs)

  model = build_model(
      input_shape=(21, data_loader.n_feature),
      pred_len=1,
      n_block=2,
      ff_dim=220,
      target_slice=data_loader.target_slice
  )

  tf.keras.utils.set_random_seed(random_seeds[i])

  optimizer = tf.keras.optimizers.Adam(0.0005)

  model.compile(optimizer, loss='mse', metrics=['mae', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error'])

  early_stop_callback = tf.keras.callbacks.EarlyStopping(
      monitor='val_loss',
      patience=5
  )

  start_training_time = time.time()

  history = model.fit(
      train_data,
      epochs= 250,
      validation_data=val_data,
      callbacks=[early_stop_callback]
  )

  end_training_time = time.time()
  elasped_training_time = end_training_time - start_training_time
  print(f'Training finished in {elasped_training_time} seconds')
  RMSE.append(np.sqrt(history.history['val_loss'][np.argmin(history.history['val_loss'])]))
  MAE.append(history.history['val_mae'][np.argmin(history.history['val_mae'])])
  MAPE.append(history.history['val_mean_absolute_percentage_error'][np.argmin(history.history['val_mean_absolute_percentage_error'])])
  QLIKE.append(history.history['val_mean_squared_logarithmic_error'][np.argmin(history.history['val_mean_squared_logarithmic_error'])])

Epoch 1/250
53/53 [==============================] - 5s 21ms/step - loss: 12.6206 - mae: 2.6571 - mean_absolute_percentage_error: 2601.5588 - mean_squared_logarithmic_error: 0.7403 - val_loss: 0.5630 - val_mae: 0.5455 - val_mean_absolute_percentage_error: 361.3782 - val_mean_squared_logarithmic_error: 0.0643
Epoch 2/250
53/53 [==============================] - 1s 16ms/step - loss: 6.5064 - mae: 1.9521 - mean_absolute_percentage_error: 2504.5286 - mean_squared_logarithmic_error: 0.5066 - val_loss: 0.4550 - val_mae: 0.4786 - val_mean_absolute_percentage_error: 328.5718 - val_mean_squared_logarithmic_error: 0.0529
Epoch 3/250
53/53 [==============================] - 1s 14ms/step - loss: 4.0891 - mae: 1.5366 - mean_absolute_percentage_error: 1502.3297 - mean_squared_logarithmic_error: 0.3648 - val_loss: 0.4049 - val_mae: 0.4479 - val_mean_absolute_percentage_error: 331.9006 - val_mean_squared_logarithmic_error: 0.0472
Epoch 4/250
53/53 [==============================] - 1s 22ms/step - loss

In [ ]:
for i in range(5):
  i=i+5*9
  data_loader = DataLoader(batch_size=32, seq_len=21, pred_len=1)

  train_data = data_loader.get_train()
  val_data = data_loader.get_val()
  test_data = data_loader.get_test()

  def res_block(inputs, ff_dim):

    norm = layers.BatchNormalization

    # Time mixing
    x = norm(axis=[-2, -1])(inputs)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(x.shape[-1], activation='relu')(x)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Input Length, Channel]
    x = layers.Dropout(0.7)(x)
    res = x + inputs

    # Feature mixing
    x = norm(axis=[-2, -1])(res)
    x = layers.Dense(ff_dim, activation='relu')(x)  # [Batch, Input Length, FF_Dim]
    x = layers.Dropout(0.7)(x)
    x = layers.Dense(inputs.shape[-1])(x)  # [Batch, Input Length, Channel]
    x = layers.Dropout(0.7)(x)
    return x + res

  def build_model(
      input_shape,
      pred_len,
      n_block,
      ff_dim,
      target_slice,
  ):

    inputs = tf.keras.Input(shape=input_shape)
    x = inputs  # [Batch, Input Length, Channel]
    for _ in range(n_block):
      x = res_block(x, ff_dim)

    if target_slice:
      x = x[:, :, target_slice]

    # Temporal projection
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(pred_len)(x)  # [Batch, Channel, Output Length]
    outputs = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Output Length, Channel])

    return tf.keras.Model(inputs, outputs)

  model = build_model(
      input_shape=(21, data_loader.n_feature),
      pred_len=1,
      n_block=2,
      ff_dim=220,
      target_slice=data_loader.target_slice
  )

  tf.keras.utils.set_random_seed(random_seeds[i])

  optimizer = tf.keras.optimizers.Adam(0.0005)

  model.compile(optimizer, loss='mse', metrics=['mae', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error'])

  early_stop_callback = tf.keras.callbacks.EarlyStopping(
      monitor='val_loss',
      patience=5
  )

  start_training_time = time.time()

  history = model.fit(
      train_data,
      epochs= 250,
      validation_data=val_data,
      callbacks=[early_stop_callback]
  )

  end_training_time = time.time()
  elasped_training_time = end_training_time - start_training_time
  print(f'Training finished in {elasped_training_time} seconds')
  RMSE.append(np.sqrt(history.history['val_loss'][np.argmin(history.history['val_loss'])]))
  MAE.append(history.history['val_mae'][np.argmin(history.history['val_mae'])])
  MAPE.append(history.history['val_mean_absolute_percentage_error'][np.argmin(history.history['val_mean_absolute_percentage_error'])])
  QLIKE.append(history.history['val_mean_squared_logarithmic_error'][np.argmin(history.history['val_mean_squared_logarithmic_error'])])

Epoch 1/250
53/53 [==============================] - 5s 22ms/step - loss: 14.8539 - mae: 2.8546 - mean_absolute_percentage_error: 4509.6309 - mean_squared_logarithmic_error: 0.7648 - val_loss: 0.6867 - val_mae: 0.6019 - val_mean_absolute_percentage_error: 449.9005 - val_mean_squared_logarithmic_error: 0.0752
Epoch 2/250
53/53 [==============================] - 1s 15ms/step - loss: 7.6193 - mae: 2.0877 - mean_absolute_percentage_error: 2247.2468 - mean_squared_logarithmic_error: 0.5341 - val_loss: 0.5486 - val_mae: 0.5317 - val_mean_absolute_percentage_error: 400.4739 - val_mean_squared_logarithmic_error: 0.0590
Epoch 3/250
53/53 [==============================] - 1s 14ms/step - loss: 5.0845 - mae: 1.6965 - mean_absolute_percentage_error: 1990.8885 - mean_squared_logarithmic_error: 0.4131 - val_loss: 0.4797 - val_mae: 0.4951 - val_mean_absolute_percentage_error: 371.1362 - val_mean_squared_logarithmic_error: 0.0512
Epoch 4/250
53/53 [==============================] - 1s 15ms/step - loss

In [ ]:
for i in range(5):
  i=i+5*10
  data_loader = DataLoader(batch_size=32, seq_len=21, pred_len=1)

  train_data = data_loader.get_train()
  val_data = data_loader.get_val()
  test_data = data_loader.get_test()

  def res_block(inputs, ff_dim):

    norm = layers.BatchNormalization

    # Time mixing
    x = norm(axis=[-2, -1])(inputs)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(x.shape[-1], activation='relu')(x)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Input Length, Channel]
    x = layers.Dropout(0.7)(x)
    res = x + inputs

    # Feature mixing
    x = norm(axis=[-2, -1])(res)
    x = layers.Dense(ff_dim, activation='relu')(x)  # [Batch, Input Length, FF_Dim]
    x = layers.Dropout(0.7)(x)
    x = layers.Dense(inputs.shape[-1])(x)  # [Batch, Input Length, Channel]
    x = layers.Dropout(0.7)(x)
    return x + res

  def build_model(
      input_shape,
      pred_len,
      n_block,
      ff_dim,
      target_slice,
  ):

    inputs = tf.keras.Input(shape=input_shape)
    x = inputs  # [Batch, Input Length, Channel]
    for _ in range(n_block):
      x = res_block(x, ff_dim)

    if target_slice:
      x = x[:, :, target_slice]

    # Temporal projection
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(pred_len)(x)  # [Batch, Channel, Output Length]
    outputs = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Output Length, Channel])

    return tf.keras.Model(inputs, outputs)

  model = build_model(
      input_shape=(21, data_loader.n_feature),
      pred_len=1,
      n_block=2,
      ff_dim=220,
      target_slice=data_loader.target_slice
  )

  tf.keras.utils.set_random_seed(random_seeds[i])

  optimizer = tf.keras.optimizers.Adam(0.0005)

  model.compile(optimizer, loss='mse', metrics=['mae', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error'])

  early_stop_callback = tf.keras.callbacks.EarlyStopping(
      monitor='val_loss',
      patience=5
  )

  start_training_time = time.time()

  history = model.fit(
      train_data,
      epochs= 250,
      validation_data=val_data,
      callbacks=[early_stop_callback]
  )

  end_training_time = time.time()
  elasped_training_time = end_training_time - start_training_time
  print(f'Training finished in {elasped_training_time} seconds')
  RMSE.append(np.sqrt(history.history['val_loss'][np.argmin(history.history['val_loss'])]))
  MAE.append(history.history['val_mae'][np.argmin(history.history['val_mae'])])
  MAPE.append(history.history['val_mean_absolute_percentage_error'][np.argmin(history.history['val_mean_absolute_percentage_error'])])
  QLIKE.append(history.history['val_mean_squared_logarithmic_error'][np.argmin(history.history['val_mean_squared_logarithmic_error'])])

Epoch 1/250
53/53 [==============================] - 6s 22ms/step - loss: 19.0818 - mae: 3.2568 - mean_absolute_percentage_error: 3434.2964 - mean_squared_logarithmic_error: 0.8125 - val_loss: 1.1522 - val_mae: 0.8272 - val_mean_absolute_percentage_error: 457.8544 - val_mean_squared_logarithmic_error: 0.1091
Epoch 2/250
53/53 [==============================] - 1s 15ms/step - loss: 10.2738 - mae: 2.4230 - mean_absolute_percentage_error: 2684.3159 - mean_squared_logarithmic_error: 0.5975 - val_loss: 0.9505 - val_mae: 0.7492 - val_mean_absolute_percentage_error: 420.9289 - val_mean_squared_logarithmic_error: 0.0872
Epoch 3/250
53/53 [==============================] - 1s 15ms/step - loss: 7.0486 - mae: 2.0001 - mean_absolute_percentage_error: 2521.1685 - mean_squared_logarithmic_error: 0.4803 - val_loss: 0.8319 - val_mae: 0.7067 - val_mean_absolute_percentage_error: 396.8659 - val_mean_squared_logarithmic_error: 0.0752
Epoch 4/250
53/53 [==============================] - 1s 14ms/step - los

In [ ]:
for i in range(5):
  i=i+5*11
  data_loader = DataLoader(batch_size=32, seq_len=21, pred_len=1)

  train_data = data_loader.get_train()
  val_data = data_loader.get_val()
  test_data = data_loader.get_test()

  def res_block(inputs, ff_dim):

    norm = layers.BatchNormalization

    # Time mixing
    x = norm(axis=[-2, -1])(inputs)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(x.shape[-1], activation='relu')(x)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Input Length, Channel]
    x = layers.Dropout(0.7)(x)
    res = x + inputs

    # Feature mixing
    x = norm(axis=[-2, -1])(res)
    x = layers.Dense(ff_dim, activation='relu')(x)  # [Batch, Input Length, FF_Dim]
    x = layers.Dropout(0.7)(x)
    x = layers.Dense(inputs.shape[-1])(x)  # [Batch, Input Length, Channel]
    x = layers.Dropout(0.7)(x)
    return x + res

  def build_model(
      input_shape,
      pred_len,
      n_block,
      ff_dim,
      target_slice,
  ):

    inputs = tf.keras.Input(shape=input_shape)
    x = inputs  # [Batch, Input Length, Channel]
    for _ in range(n_block):
      x = res_block(x, ff_dim)

    if target_slice:
      x = x[:, :, target_slice]

    # Temporal projection
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(pred_len)(x)  # [Batch, Channel, Output Length]
    outputs = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Output Length, Channel])

    return tf.keras.Model(inputs, outputs)

  model = build_model(
      input_shape=(21, data_loader.n_feature),
      pred_len=1,
      n_block=2,
      ff_dim=220,
      target_slice=data_loader.target_slice
  )

  tf.keras.utils.set_random_seed(random_seeds[i])

  optimizer = tf.keras.optimizers.Adam(0.0005)

  model.compile(optimizer, loss='mse', metrics=['mae', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error'])

  early_stop_callback = tf.keras.callbacks.EarlyStopping(
      monitor='val_loss',
      patience=5
  )

  start_training_time = time.time()

  history = model.fit(
      train_data,
      epochs= 250,
      validation_data=val_data,
      callbacks=[early_stop_callback]
  )

  end_training_time = time.time()
  elasped_training_time = end_training_time - start_training_time
  print(f'Training finished in {elasped_training_time} seconds')
  RMSE.append(np.sqrt(history.history['val_loss'][np.argmin(history.history['val_loss'])]))
  MAE.append(history.history['val_mae'][np.argmin(history.history['val_mae'])])
  MAPE.append(history.history['val_mean_absolute_percentage_error'][np.argmin(history.history['val_mean_absolute_percentage_error'])])
  QLIKE.append(history.history['val_mean_squared_logarithmic_error'][np.argmin(history.history['val_mean_squared_logarithmic_error'])])

Epoch 1/250
53/53 [==============================] - 6s 25ms/step - loss: 20.1414 - mae: 3.3043 - mean_absolute_percentage_error: 3931.7424 - mean_squared_logarithmic_error: 0.9532 - val_loss: 0.7986 - val_mae: 0.6832 - val_mean_absolute_percentage_error: 369.2306 - val_mean_squared_logarithmic_error: 0.0872
Epoch 2/250
53/53 [==============================] - 1s 16ms/step - loss: 10.7271 - mae: 2.4905 - mean_absolute_percentage_error: 3758.0962 - mean_squared_logarithmic_error: 0.6398 - val_loss: 0.5866 - val_mae: 0.5649 - val_mean_absolute_percentage_error: 344.5538 - val_mean_squared_logarithmic_error: 0.0689
Epoch 3/250
53/53 [==============================] - 1s 19ms/step - loss: 7.0525 - mae: 2.0265 - mean_absolute_percentage_error: 2952.2476 - mean_squared_logarithmic_error: 0.5100 - val_loss: 0.4598 - val_mae: 0.4780 - val_mean_absolute_percentage_error: 349.2249 - val_mean_squared_logarithmic_error: 0.0564
Epoch 4/250
53/53 [==============================] - 1s 15ms/step - los

In [ ]:
np.argmin(RMSE)

16

In [ ]:
np.argmin(MAE)

13

In [ ]:
np.argmin(MAPE)

19

In [ ]:
np.argmin(QLIKE)

16

In [ ]:
random_seeds[1+5*11]

89102543

# Test (main sample)

In [ ]:
class DataLoader:

  def __init__(self, batch_size, seq_len, pred_len):
    self.batch_size = batch_size
    self.seq_len = seq_len
    self.pred_len = pred_len
    self.target_slice = slice(0, None)

    self._read_data()

  def _read_data(self):

    filepath = ('SP100_vol (original).xlsx')

    df_raw = pd.read_excel(filepath)
    df = df_raw.set_index('Date')

    # split train/valid/test
    n = len(df)
    train_end = int(n * 0.7)-5
    val_end = int(n * 0.7)+5
    test_end = n

    train_df = df[:train_end]
    val_df = df[train_end - self.seq_len : val_end]
    test_df = df[val_end - self.seq_len : test_end]

    # standardize by training set
    self.scaler = StandardScaler()
    self.scaler.fit(train_df.values)

    def scale_df(df, scaler):
      data = scaler.transform(df.values)
      return pd.DataFrame(data, index=df.index, columns=df.columns)

    self.train_df = scale_df(train_df, self.scaler)
    self.val_df = scale_df(val_df, self.scaler)
    self.test_df = scale_df(test_df, self.scaler)
    self.n_feature = self.train_df.shape[-1]

  def _split_window(self, data):
    inputs = data[:, : self.seq_len, :]
    labels = data[:, self.seq_len :, self.target_slice]

    inputs.set_shape([None, self.seq_len, None])
    labels.set_shape([None, self.pred_len, None])
    return inputs, labels

  def _make_dataset(self, data, shuffle=False):
    data = np.array(data, dtype=np.float32)
    ds = tf.keras.utils.timeseries_dataset_from_array(
        data=data,
        targets=None,
        sequence_length=(self.seq_len + self.pred_len),
        sequence_stride=1,
        shuffle=shuffle,
        batch_size=self.batch_size,
    )
    ds = ds.map(self._split_window)
    return ds

  def inverse_transform(self, data):
    return self.scaler.inverse_transform(data)

  def get_train(self, shuffle=True):
    return self._make_dataset(self.train_df, shuffle=shuffle)

  def get_val(self):
    return self._make_dataset(self.val_df, shuffle=False)

  def get_test(self):
    return self._make_dataset(self.test_df, shuffle=False)

  def make_forecast(self, model):
    test_data = self.test_df.values
    predictions = []

    for i in range(0, len(test_data) - self.seq_len, self.pred_len):
        input_seq = test_data[i:i + self.seq_len]
        input_seq = input_seq.reshape((1, self.seq_len, -1))

            # Make prediction and reshape output
        pred = model.predict(input_seq)
        pred = pred.reshape(self.pred_len, -1)

        predictions.append(pred)

        # Concatenate all predictions
    predictions = np.concatenate(predictions, axis=0)
    return predictions

In [ ]:
  data_loader = DataLoader(batch_size=32, seq_len=21, pred_len=1)

  train_data = data_loader.get_train()
  val_data = data_loader.get_val()
  test_data = data_loader.get_test()

  def res_block(inputs, ff_dim):

    norm = layers.BatchNormalization

    # Time mixing
    x = norm(axis=[-2, -1])(inputs)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(x.shape[-1], activation='relu')(x)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Input Length, Channel]
    x = layers.Dropout(0.7)(x)
    res = x + inputs

    # Feature mixing
    x = norm(axis=[-2, -1])(res)
    x = layers.Dense(ff_dim, activation='relu')(x)  # [Batch, Input Length, FF_Dim]
    x = layers.Dropout(0.7)(x)
    x = layers.Dense(inputs.shape[-1])(x)  # [Batch, Input Length, Channel]
    x = layers.Dropout(0.7)(x)
    return x + res

  def build_model(
      input_shape,
      pred_len,
      n_block,
      ff_dim,
      target_slice,
  ):

    inputs = tf.keras.Input(shape=input_shape)
    x = inputs  # [Batch, Input Length, Channel]
    for _ in range(n_block):
      x = res_block(x, ff_dim)

    if target_slice:
      x = x[:, :, target_slice]

    # Temporal projection
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(pred_len)(x)  # [Batch, Channel, Output Length]
    outputs = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Output Length, Channel])

    return tf.keras.Model(inputs, outputs)

  model = build_model(
      input_shape=(21, data_loader.n_feature),
      pred_len=1,
      n_block=2,
      ff_dim=220,
      target_slice=data_loader.target_slice
  )

  tf.keras.utils.set_random_seed(89102543)

  optimizer = tf.keras.optimizers.Adam(0.0005)

  model.compile(optimizer, loss='mse', metrics=['mae', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error'])

  #early_stop_callback = tf.keras.callbacks.EarlyStopping(
  #    monitor='val_loss',
  #    patience=140
  #)

  start_training_time = time.time()

  history = model.fit(
      train_data,
      epochs= 168 #accounting for early stop
      #validation_data=val_data,
     # callbacks=[early_stop_callback]
  )

  end_training_time = time.time()
  elasped_training_time = end_training_time - start_training_time
  print(f'Training finished in {elasped_training_time} seconds')

Epoch 1/168
74/74 [==============================] - 5s 11ms/step - loss: 14.1830 - mae: 2.7335 - mean_absolute_percentage_error: 3582.8064 - mean_squared_logarithmic_error: 0.7346
Epoch 2/168
74/74 [==============================] - 1s 11ms/step - loss: 6.7499 - mae: 1.9494 - mean_absolute_percentage_error: 3456.7544 - mean_squared_logarithmic_error: 0.5053
Epoch 3/168
74/74 [==============================] - 1s 11ms/step - loss: 4.3253 - mae: 1.5547 - mean_absolute_percentage_error: 4053.6653 - mean_squared_logarithmic_error: 0.3785
Epoch 4/168
74/74 [==============================] - 1s 17ms/step - loss: 3.1387 - mae: 1.3242 - mean_absolute_percentage_error: 3659.1370 - mean_squared_logarithmic_error: 0.3062
Epoch 5/168
74/74 [==============================] - 1s 16ms/step - loss: 2.3498 - mae: 1.1364 - mean_absolute_percentage_error: 2270.6006 - mean_squared_logarithmic_error: 0.2482
Epoch 6/168
74/74 [==============================] - 1s 11ms/step - loss: 1.8268 - mae: 1.0014 - me

In [ ]:
import tensorflow as tf
import numpy as np

# Define the path to the saved model
model_path = '/content/saved_model.pb'  # Update with your model path in Colab

# Load the model using tf.saved_model.load
loaded_model = tf.saved_model.load('my_model.h5')

OSError: SavedModel file does not exist at: my_model.h5/{saved_model.pbtxt|saved_model.pb}

In [ ]:
predictions = model.predict(test_data)
scaled_preds = predictions
scaled_preds=scaled_preds.reshape(scaled_preds.shape[0], scaled_preds.shape[2])
scaled_preds.shape
preds = data_loader.inverse_transform(scaled_preds)
RMSE_test=np.sqrt(((test_true.iloc[5:]-preds)**2).sum(axis=1).sum(axis=0)/(80*len(test_true["NKE"])))
MAE_test=((abs(test_true.iloc[5:]-preds)).sum(axis=1).sum(axis=0)/(80*len(test_true["NKE"])))
MAPE_test=((abs(test_true.iloc[5:]-preds)/test_true.iloc[5:]).sum(axis=1).sum(axis=0)/(80*len(test_true["NKE"])))
QLIKE_test=((test_true.iloc[5:]/preds)-np.log(test_true.iloc[5:]/preds)-1).sum(axis=1).sum(axis=0)/(80*len(test_true["NKE"]))
print(RMSE_test)
print(MAE_test)
print(MAPE_test)
print(QLIKE_test)

32/32 [==============================] - 0s 8ms/step
0.0036982268151303658
0.0023926737111054095
0.19080390357461194
0.030349418715220736


In [ ]:
test_true=test_true.iloc[5:]
test_true.iloc[:]=preds
test_true.head()

<ipython-input-30-860a8db6f4fc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_true.iloc[:]=preds


,AMT,COP,CVX,AXP,BAC,C,CB,CME,GS,JPM,...,TMO,UNH,AMZN,HD,LOW,MCD,NKE,SBUX,TGT,TJX
Date,,,,,,,,,,,,,,,,,,,,,
2017-04-27,0.006867,0.014668,0.008954,0.007900,0.012896,0.011017,0.008337,0.008665,0.010140,0.008647,...,0.011537,0.007676,0.007680,0.007453,0.008256,0.007448,0.008088,0.008011,0.008444,0.007077
2017-04-28,0.009830,0.013619,0.008120,0.007459,0.011754,0.010606,0.006164,0.011428,0.009251,0.008155,...,0.008777,0.008252,0.007861,0.006982,0.008018,0.007127,0.008203,0.008213,0.007832,0.007333
2017-05-01,0.008513,0.012210,0.008185,0.007756,0.011055,0.009828,0.005758,0.009271,0.009231,0.007549,...,0.007330,0.007474,0.010176,0.007198,0.008008,0.006119,0.007310,0.009921,0.008063,0.007182
2017-05-02,0.008134,0.011778,0.007641,0.007521,0.011853,0.010624,0.005827,0.008215,0.009977,0.008303,...,0.007994,0.007778,0.009222,0.006479,0.008162,0.006096,0.007929,0.009009,0.007739,0.007550
2017-05-03,0.008078,0.016837,0.007394,0.007864,0.010872,0.009597,0.005518,0.008130,0.009521,0.007499,...,0.006723,0.012209,0.008493,0.006677,0.008183,0.006544,0.007127,0.007742,0.007667,0.007389


In [ ]:
test_true.to_excel("TSMixer forecasts.xlsx")